In [1]:
import pandas as pd
import random

In [72]:
results = pd.read_csv('results.csv')
results.loc[:, 'mask'] = results.loc[:, 'mask'].str.replace('X', '')
roster = pd.read_csv('roster.csv')
tournaments = pd.read_csv('tournaments.csv').set_index('idtournament')

In [73]:
# correct number of questions in tournaments
results.groupby('idtournament').first()

,idteam,current_name,base_name,position,questions_total,mask,bonus_a,bonus_b,tech_rating,predicted_position,d_bonus_a,d_bonus_b,d_diff_bonus,included_in_rating,diff_bonus
idtournament,,,,,,,,,,,,,,,
2,53,Анталлактика,Анталлактика,11.0,0,000000000000000,369.0,72.0,1151.0,15.0,369,272,72.0,0.0,NaN
3,1,Неспроста,Неспроста,6.0,0,000000000000000,1803.0,-159.0,10442.0,3.0,1803,1603,-159.0,0.0,NaN
6,10,Яровит,Яровит,21.0,0,000000000000000,187.0,-55.0,1099.0,13.0,187,88,-55.0,0.0,NaN
8,1,Неспроста,Неспроста,4.0,0,000000000000000,2032.0,237.0,10082.0,5.0,2032,2017,237.0,0.0,NaN
9,7,Ра,Ра,5.0,0,000000000000000,1028.0,-213.0,8453.0,1.0,1028,503,-213.0,0.0,NaN
11,2,Афина,Афина,2.0,0,000000000000000,1853.0,-15.0,11965.0,1.0,1853,1918,-15.0,0.0,NaN
12,1,Неспроста,Неспроста,1.0,0,000000000000000,2297.0,339.0,10319.0,4.0,2297,2293,339.0,0.0,NaN
14,3,Команда Ильи Иткина,Команда Ильи Иткина,12.0,0,NaN,570.0,487.0,9126.0,1.0,570,487,-532.0,NaN,-532.0
15,13,Дельта фи,Дельта фи,8.0,0,000000000000000,874.0,57.0,2283.0,14.0,874,547,57.0,0.0,NaN


In [56]:
tournaments = tournaments.loc[~tournaments.questions_total.isnull(), :]
tournaments.head(1)

,name,town,long_name,date_start,date_end,tour_count,tour_questions,tour_ques_per_tour,questions_total,type_name,main_payment_value,discounted_payment_value,discounted_payment_reason,date_requests_allowed_to,comment,site_url,main_payment_currency,discounted_payment_currency
idtournament,,,,,,,,,,,,,,,,,,
3236,Десятый блин,Великий Новгород,"Открытый Фестиваль интеллектуальных игр ""Десят...",2015-05-23 13:00:00,2015-05-24 16:00:00,6.0,12.0,NaN,72.0,Обычный,2500.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
results.head(2)

,idteam,current_name,base_name,position,questions_total,mask,bonus_a,bonus_b,tech_rating,predicted_position,d_bonus_a,d_bonus_b,d_diff_bonus,included_in_rating,idtournament,diff_bonus
0,209,Команда Коваленко,Команда Коваленко,11.0,34,1111101000101111001100011000111101001111001110...,2230.0,-87.0,4509.0,7.0,2230,663,-87.0,1.0,3236,NaN
1,270,Синоп,Синоп,14.5,31,1110110000101100001100101101101101010011010000...,2189.0,119.0,2567.0,20.0,2189,618,119.0,1.0,3236,NaN


In [33]:
tour = random.sample(results.idtournament.unique().tolist(), 1)[0]
t_subset = results.loc[results.idtournament == tour, ['idteam', 'mask']]
team = random.sample(t_subset.idteam.tolist(), 1)[0]
quest = random.randint(0, int(tournaments.loc[tour, 'questions_total']))
quest_key = f"{str(tour)}_{str(quest)}"

In [35]:
question_difficulty[quest_key]

5.475996237008928

In [32]:
def difficulty_init():
    return random.uniform(0, 10)
question_difficulty = {f"{str(tourn)}_{str(ques)}": difficulty_init() for tourn in tournaments.index 
                       for ques in range(int(tournaments.loc[tourn, 'questions_total']))}
pd.DataFrame(question_difficulty, index=[0]).T.head()

,0
3236_0,5.124221
3236_1,7.166873
3236_2,8.385169
3236_3,7.186965
3236_4,9.222628


In [53]:
def sigmoid(x):
    return 1 / (1 + pd.np.exp(-x))    

In [36]:
def gd(lr=0.01, epochs=10):
    # get random tournament, team and question
    tour = random.sample(results.idtournament.unique().tolist(), 1)[0]
    t_subset = results.loc[results.idtournament == tour, ['idteam', 'mask']]
    team = random.sample(t_subset.idteam.tolist(), 1)[0]
    players = roster.loc[(roster.idteam == team) & (roster.idtournament == tour), 'idplayer'].tolist()
    quest = random.randint(0, int(tournaments.loc[tour, 'questions_total']))
    
    # get question difficulty
    quest_dif_key = f"{str(tour)}_{str(quest)}"
    difficulty = question_difficulty[quest_dif_key]
    
    # calculate team skill and sigmoid
    skills = [player_skills_dict[player] for player in players]
    team_skill = sum(skills)
    team_sigmoid = sigmoid(team_skill - difficulty)
    
    # get y (1 for correct answer, 0 for incorrect)
    y_true = int(t_subset.loc[t_subset.idteam == team, 'mask'].values[0][quest])
    
    # weights update
    question_difficulty[quest_dif_key] += lr * (y_true * (1-team_sigmoid) + (1-y)*team_sigmoid)
    for 

SyntaxError: invalid syntax (<ipython-input-36-3ec95872f80e>, line 7)

In [71]:
t_subset.loc[:, 'mask'].str.replace('X', '')

90986    0000000110000010010010111010010000000000000010...
90987    0011011111110111111100111111100111011011011110...
90988    0111111101110111011101110000010110110011101110...
90989    0101011111110010011110110110010111111111101111...
90990    0101011111111100011001111011100110001011111110...
90991    0111110111111111111001100110100110111111111110...
90992    0111000111000110011000110011000000110000001000...
90993    0011100101100101011100111011010111100011111100...
90994    0101010101010110011001110010000110011110101000...
90995    0101000100100000011000000000000000100000001000...
90996    0110100100010010011000011000000000100000011010...
90997    0001000101100011011001010001010000100000001001...
90998    0101110111101111011001110110100111011011101010...
90999    0000000100000000010001000000000010100000000011...
91000    0010000010000010001000010000000000000000000000...
91001    0111110001010000011000110010100110000000000000...
91002    0101100111001011011010110011000100100000101001.

In [67]:
t_subset

,idteam,mask
90986,53,00000001100000100XXXXXXX100101110100100000XXXX...
90987,55,00110111111101111XXXXXXX111001111111001110XXXX...
90988,67,01111111011101110XXXXXXX111011100000101101XXXX...
90989,185,01010111111100100XXXXXXX111101101100101111XXXX...
90990,264,01010111111111000XXXXXXX110011110111001100XXXX...
90991,313,01111101111111111XXXXXXX110011001101001101XXXX...
90992,683,01110001110001100XXXXXXX110001100110000001XXXX...
90993,791,00111001011001010XXXXXXX111001110110101111XXXX...
90994,1021,01010101010101100XXXXXXX110011100100001100XXXX...
90995,1093,01010001001000000XXXXXXX110000000000000001XXXX...


In [63]:
len(results.loc[results.idtournament == tour, 'mask'].values[0])

72